In [2]:
import pandas as pd
import os

In [7]:
def getXY(ids,labels,concat_cols):
    data_icu = True
    concat = False
    X_df=pd.DataFrame()   
    y_df=pd.DataFrame()   
    features=[]
    #print(ids)
    # For every one of the samples in the ./data/csv folder, get the data. 
    for sample in ids:
        # Get different labels depending on whether or not we are looking at ICU data. 
        if data_icu:
            y=labels[labels['stay_id']==sample]['label']
        else:
            y=labels[labels['hadm_id']==sample]['label']

        # Read in the dynamic data. 
        dyn=pd.read_csv('./data/csv/'+str(sample)+'/dynamic.csv',header=[0,1])

        # I do not know what this concat stuff is doing, but I believe it is not what we ae working on. 
        if concat:
            dyn.columns=dyn.columns.droplevel(0)
            dyn=dyn.to_numpy()
            dyn=dyn.reshape(1,-1)
            #print(dyn.shape)
            #print(len(concat_cols))
            dyn_df=pd.DataFrame(data=dyn,columns=concat_cols)
            features=concat_cols

        # This is the only one we are really going to use, for now. 
        else:
            dyn_df=pd.DataFrame()
            #print(dyn)
            for key in dyn.columns.levels[0]:
                #print(sample)                    
                dyn_temp=dyn[key]
                # This is the one that we are using. 
                if data_icu:
                    if ((key=="CHART") or (key=="MEDS")):
                        agg=dyn_temp.aggregate("mean")
                        agg=agg.reset_index()
                    else:
                        agg=dyn_temp.aggregate("max")
                        agg=agg.reset_index()
                else:
                    if ((key=="LAB") or (key=="MEDS")):
                        agg=dyn_temp.aggregate("mean")
                        agg=agg.reset_index()
                    else:
                        agg=dyn_temp.aggregate("max")
                        agg=agg.reset_index()
                if dyn_df.empty:
                    dyn_df=agg
                else:
                    dyn_df=pd.concat([dyn_df,agg],axis=0)
            dyn_df=dyn_df.T
            dyn_df.columns = dyn_df.iloc[0]
            dyn_df=dyn_df.iloc[1:,:]
        # get the static and demo data. 
        stat=pd.read_csv('./data/csv/'+str(sample)+'/static.csv',header=0)
        demo=pd.read_csv('./data/csv/'+str(sample)+'/demo.csv',header=0)

        # Fill up the X_df with the demo data. 
        if X_df.empty:
            X_df=pd.concat([dyn_df,stat],axis=1)
            X_df=pd.concat([X_df,demo],axis=1)
        else:
            X_df=pd.concat([X_df,pd.concat([pd.concat([dyn_df,stat],axis=1),demo],axis=1)],axis=0)

        # Just make y equal to the label (with the labels describing how many 
        if y_df.empty:
            y_df=y
        else:
            y_df=pd.concat([y_df,y],axis=0)
            
    # remove all of the duplicate columns. 
    X_df = X_df.loc[:,~X_df.columns.duplicated()].copy()
    print("X_df",X_df.shape)
    print("y_df",y_df.shape)
    return X_df ,y_df

In [8]:
ids = [30001471, 30005366, 30009505]
concat_cols = None
labels=pd.read_csv('./data/csv/labels.csv', header=0)

In [9]:
X_df, Y_df = getXY(ids,labels,concat_cols)

X_df (3, 17)
y_df (3,)


In [10]:
print(X_df)

      220045 220051 220052 220061 220245  subject_id   stay_id  \
0  79.138889    0.0    0.0    0.0    0.0    17461994  30001471   
0  77.819444    0.0    0.0    0.0    0.0    18855147  30005366   
0  84.298611    0.0    0.0    0.0    0.0    12780478  30009505   

                intime              outtime  Age gender  \
0  2189-05-02 06:57:00  2189-05-05 16:02:33   86      M   
0  2202-12-27 17:36:59  2202-12-31 14:30:31   69      F   
0  2155-07-02 19:56:14  2155-07-15 17:06:07   69      M   

                      ethnicity insurance  label   hadm_id  los  select_time  
0  HISPANIC/LATINO - SALVADORAN     Other      0  29537385   72            9  
0                         WHITE  Medicare      0  25702658   72           20  
0                         WHITE  Medicare      0  28468774   72          237  


In [11]:
print(Y_df)

2027    0
2299    0
1025    0
Name: label, dtype: int64
